In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
import os
os.chdir(r"C:\Users\cdron\Desktop\Cleanup\Misc\kaggle\Search_results_Relevance")
os.getcwd()

'C:\\Users\\cdron\\Desktop\\Cleanup\\Misc\\kaggle\\Search_results_Relevance'

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
# we dont need ID columns
idx = test.id.values.astype(int)
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [5]:
# create labels. drop useless columns
y = train.median_relevance.values
train = train.drop(['median_relevance', 'relevance_variance'], axis=1)

In [6]:
# do some lambda magic on text columns
traindata = list(train.apply(lambda x:'%s %s %s' % (x['query'],x['product_title'], x['product_description']),axis=1))
testdata = list(test.apply(lambda x:'%s %s %s' % (x['query'],x['product_title'], x['product_description']),axis=1))

In [7]:
# the infamous tfidf vectorizer (Do you remember this one?)
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
        strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
        ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
        stop_words = 'english')

In [8]:
# Fit TFIDF
tfv.fit(traindata)
X =  tfv.transform(traindata) 
X_test = tfv.transform(testdata)

In [20]:
model = LogisticRegression(penalty='l2', dual=True, tol=0.00001,
                           C=1.1, fit_intercept=True, intercept_scaling=1.0,
                           class_weight='auto', random_state=44)

In [21]:
# Fit Logistic Regression Model
model.fit(X, y)
preds = model.predict(X_test)

In [22]:
# Create your first submission file
submission = pd.DataFrame({"id": idx, "prediction": preds})
submission.to_csv("code_benchmark_lr_regular_loose.csv", index=False)